<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/Fault_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Initialization and Firebase Setup

This section prepares the project by cloning the repository, ensuring it's on the correct branch, and loading Firebase-related logic and rules for use in the notebook.

In [ ]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))  # for FireBase.ipynb
sys.path.append(REPO_DIR)                            # for FAULT_RULES.ipynb
from importnb import Notebook

with Notebook():
    import FireBase as fb                           # now fb.add_user, fb.get_user, etc. are available
    from FAULT_RULES import FAULT_RULES as fr       # now FAULT_RULES are aviable to use


firebase_url = fb.firebase_url

# Fault Controller

## Detect and Log Active Faults from Sensor Data

retrieves real-time indoor and outdoor sensor data from Firebase and compares each reading against predefined thresholds from `FAULT_RULES`. If a value falls outside the allowed range and hasn’t been previously logged, a new fault is created in the active faults database.


In [ ]:
import requests
import json
from datetime import datetime

def process_faults_from_database():
    active_faults = fb.get_active_faults()
    existing_titles = set()

    if isinstance(active_faults, dict):
        existing_titles = {
            fault["title"]
            for fault in active_faults.values()
            if isinstance(fault, dict) and "title" in fault
        }

    location_map = {
        "IndoorData": "Indoor",
        "OutdoorData": "Outdoor"
    }

    for folder, location in location_map.items():
        url = f"{firebase_url}SensorsData/{folder}.json"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to load {folder}")
            continue

        data = response.json()
        if not isinstance(data, dict):
            print(f"No valid data in {folder}")
            continue

        for reading in data.values():
            if not isinstance(reading, dict):
                continue

            for sensor_type, value in reading.items():
                key = f"{sensor_type.lower()}_{location.lower()}"
                for rule in fr.get(key, []):
                    min_val = rule.get("min")
                    max_val = rule.get("max")

                    if (min_val is None or value >= min_val) and (max_val is None or value < max_val):
                        if rule["title"] in existing_titles:
                            continue

                        title_key = f"{rule['title']} ({datetime.now().strftime('%d-%m-%Y %H:%M:%S')})"

                        fault = {
                            "sensor": rule["sensor"],
                            "severity": rule["severity"],
                            "title": rule["title"],
                            "value": value,
                            "status": "Active",
                            "repaired_by": "",
                            "timestamp": datetime.now().isoformat(),
                            "actions": rule["actions"]
                        }

                        fb.add_active_fault(title_key, fault)
                        existing_titles.add(rule["title"])
                        print(f"[{location}] Fault Created: {rule['title']} (Severity: {rule['severity']})")

process_faults_from_database()